In [2]:
with open('assignment2dataset/train.txt', 'r') as file:
    train_data = [line.strip().split() for line in file]

In [3]:
len(train_data)

220663

In [5]:
print(train_data[0])

['Confidence', 'NN', 'B-NP']


In [6]:
with open('assignment2dataset/test.txt', 'r') as file:
    test_data = [line.strip().split() for line in file]

In [7]:
len(test_data)

49389

In [8]:
!pip install sklearn-crfsuite

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [9]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [10]:
nltk.download('conll2000')

[nltk_data] Downloading package conll2000 to /home/manas/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!


True

In [11]:
nltk.corpus.conll2000.iob_sents()

[[('Confidence', 'NN', 'B-NP'), ('in', 'IN', 'B-PP'), ('the', 'DT', 'B-NP'), ('pound', 'NN', 'I-NP'), ('is', 'VBZ', 'B-VP'), ('widely', 'RB', 'I-VP'), ('expected', 'VBN', 'I-VP'), ('to', 'TO', 'I-VP'), ('take', 'VB', 'I-VP'), ('another', 'DT', 'B-NP'), ('sharp', 'JJ', 'I-NP'), ('dive', 'NN', 'I-NP'), ('if', 'IN', 'B-SBAR'), ('trade', 'NN', 'B-NP'), ('figures', 'NNS', 'I-NP'), ('for', 'IN', 'B-PP'), ('September', 'NNP', 'B-NP'), (',', ',', 'O'), ('due', 'JJ', 'B-ADJP'), ('for', 'IN', 'B-PP'), ('release', 'NN', 'B-NP'), ('tomorrow', 'NN', 'B-NP'), (',', ',', 'O'), ('fail', 'VB', 'B-VP'), ('to', 'TO', 'I-VP'), ('show', 'VB', 'I-VP'), ('a', 'DT', 'B-NP'), ('substantial', 'JJ', 'I-NP'), ('improvement', 'NN', 'I-NP'), ('from', 'IN', 'B-PP'), ('July', 'NNP', 'B-NP'), ('and', 'CC', 'I-NP'), ('August', 'NNP', 'I-NP'), ("'s", 'POS', 'B-NP'), ('near-record', 'JJ', 'I-NP'), ('deficits', 'NNS', 'I-NP'), ('.', '.', 'O')], [('Chancellor', 'NNP', 'O'), ('of', 'IN', 'B-PP'), ('the', 'DT', 'B-NP'), ('Ex

In [16]:
train_sents = list(nltk.corpus.conll2000.iob_sents('train.txt'))
test_sents = list(nltk.corpus.conll2000.iob_sents('test.txt'))

In [17]:
len(train_sents)

8936

In [18]:
def word2features(w):
    word = w[0]
    postag = w[1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = w[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(word):
    return [word2features(word)]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [21]:
print(train_sents[0])

[('Confidence', 'NN', 'B-NP'), ('in', 'IN', 'B-PP'), ('the', 'DT', 'B-NP'), ('pound', 'NN', 'I-NP'), ('is', 'VBZ', 'B-VP'), ('widely', 'RB', 'I-VP'), ('expected', 'VBN', 'I-VP'), ('to', 'TO', 'I-VP'), ('take', 'VB', 'I-VP'), ('another', 'DT', 'B-NP'), ('sharp', 'JJ', 'I-NP'), ('dive', 'NN', 'I-NP'), ('if', 'IN', 'B-SBAR'), ('trade', 'NN', 'B-NP'), ('figures', 'NNS', 'I-NP'), ('for', 'IN', 'B-PP'), ('September', 'NNP', 'B-NP'), (',', ',', 'O'), ('due', 'JJ', 'B-ADJP'), ('for', 'IN', 'B-PP'), ('release', 'NN', 'B-NP'), ('tomorrow', 'NN', 'B-NP'), (',', ',', 'O'), ('fail', 'VB', 'B-VP'), ('to', 'TO', 'I-VP'), ('show', 'VB', 'I-VP'), ('a', 'DT', 'B-NP'), ('substantial', 'JJ', 'I-NP'), ('improvement', 'NN', 'I-NP'), ('from', 'IN', 'B-PP'), ('July', 'NNP', 'B-NP'), ('and', 'CC', 'I-NP'), ('August', 'NNP', 'I-NP'), ("'s", 'POS', 'B-NP'), ('near-record', 'JJ', 'I-NP'), ('deficits', 'NNS', 'I-NP'), ('.', '.', 'O')]


In [22]:
# X_train = [sent2features(s) for s in train_data]
# y_train = [sent2labels(s) for s in train_data]

# X_test = [sent2features(s) for s in test_data]
# y_test = [sent2labels(s) for s in test_data]


In [56]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label[0] for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]
sent2features(train_sents[1])

[{'bias': 1.0,
  'word.lower()': 'chancellor',
  'word[-3:]': 'lor',
  'word[-2:]': 'or',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'NNP',
  'postag[:2]': 'NN',
  'BOS': True,
  '+1:word.lower()': 'of',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'IN',
  '+1:postag[:2]': 'IN'},
 {'bias': 1.0,
  'word.lower()': 'of',
  'word[-3:]': 'of',
  'word[-2:]': 'of',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'IN',
  'postag[:2]': 'IN',
  '-1:word.lower()': 'chancellor',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:postag': 'NNP',
  '-1:postag[:2]': 'NN',
  '+1:word.lower()': 'the',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'DT',
  '+1:postag[:2]': 'DT'},
 {'bias': 1.0,
  'word.lower()': 'the',
  'word[-3:]': 'the',
  'word[-2:]': 'he',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False

In [29]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]


In [40]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/home/manas/.local/lib/python3.6/site-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_transitions=True, c1=0.1, c2=0.1,
    keep_tempfiles=None, max_iterations=100)

In [41]:
labels = list(crf.classes_)
#labels.remove('O')
labels

['B', 'I', 'O']

In [42]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.9481260685841246

In [43]:
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))


              precision    recall  f1-score   support

           B      0.960     0.954     0.957     23852
           I      0.932     0.938     0.935     17345
           O      0.948     0.952     0.950      6180

    accuracy                          0.948     47377
   macro avg      0.947     0.948     0.947     47377
weighted avg      0.948     0.948     0.948     47377



In [55]:
# # define fixed parameters and parameters to search
# crf = sklearn_crfsuite.CRF(
#     algorithm='lbfgs',
#     max_iterations=100,
#     all_possible_transitions=True
# )
# params_space = {
#     'c1': scipy.stats.expon(scale=0.5),
#     'c2': scipy.stats.expon(scale=0.05),
# }

# # use the same metric for evaluation
# f1_scorer = make_scorer(metrics.flat_f1_score,
#                         average='weighted', labels=labels)

# # search
# rs = RandomizedSearchCV(crf, params_space,
#                         cv=3,
#                         verbose=1,
#                         n_jobs=-1,
#                         n_iter=50,
#                         scoring=f1_scorer)
# rs.fit(X_train, y_train)

In [45]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(5))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-4:])

Top likely transitions:
O      -> O       2.090910
B      -> I       1.964276
I      -> I       1.085829
O      -> B       0.601540
B      -> B       0.369312

Top unlikely transitions:
B      -> O       -0.263557
I      -> B       -1.151294
I      -> O       -1.825871
O      -> I       -12.267853


In [46]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
7.742169 B        BOS
7.366682 I        -1:word.lower():'ve
6.476407 I        -1:word.lower():trying
6.388363 O        BOS
6.378277 O        word.lower():n't
6.317444 I        -1:word.lower():interbank
6.070636 O        -1:word.lower():says
6.065570 I        -1:word.lower():capital-gains
5.928229 I        -1:word.lower():tens
5.524759 I        -1:word.lower():vice
5.490279 I        -1:word.lower():intends
5.439327 B        -1:word.lower():able
5.400217 I        -1:word.lower():an
4.888253 I        word.lower():million
4.696904 I        -1:word.lower():tend
4.686398 I        -1:word.lower():because
4.658433 I        -1:word.lower():refuse
4.639044 I        -1:word.lower():very
4.560090 O        word.lower():themselves
4.518404 B        word.lower():something
4.499100 B        -1:word.lower():for
4.454665 I        -1:word.lower():mr.
4.425921 I        word.lower():billion
4.401573 B        -1:word.lower():455,000
4.386328 B        -1:word.lower():criminality
4.378680 O     

In [52]:
y_pred = crf.predict(X_test)
print(sklearn_crfsuite.metrics.flat_classification_report(y_test, y_pred, labels=labels))
print(sklearn_crfsuite.metrics.sequence_accuracy_score(y_test, y_pred))
print(sklearn_crfsuite.metrics.flat_accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           B       0.96      0.95      0.96     23852
           I       0.93      0.94      0.93     17345
           O       0.95      0.95      0.95      6180

    accuracy                           0.95     47377
   macro avg       0.95      0.95      0.95     47377
weighted avg       0.95      0.95      0.95     47377

0.43588469184890655
0.948097177955548
